#### Descargando la data 

Descargando los datos para el entrenamiento de los modelos.

#### Módulos

In [256]:
# Módulos
import ee
import folium
import geehydro

In [257]:
# Inicio de API
ee.Initialize()

#### Función generateGreedForGeometry

In [258]:
#funcion para la generacion de grillas
def generateGreedForGeometry(geom,dx,dy):
    """
    Función para generar grillas partiendo de una zona geográfica.
    """

    bounds = ee.Geometry(geom).bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    xmin = ll.get(0)
    xmax = ur.get(0)
    ymin = ll.get(1)
    ymax = ur.get(1)

    xx = ee.List.sequence(xmin, xmax, dx)
    yy = ee.List.sequence(ymin, ymax, dy)

    #************************************
    def contYYa(y,x,dx,dy):

        x1 = ee.Number(x)
        x2 = ee.Number(x).add(ee.Number(dx))
        y1 = ee.Number(y)
        y2 = ee.Number(y).add(ee.Number(dy))

        coords = ee.List([x1, y1, x2, y2])
        rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)

        return ee.Feature(rect)

    def contYY(x):
        return yy.map(lambda yi: contYYa(y=yi,x=x,dx=dx,dy=dy) )
    #************************************

    cells = xx.map(contYY).flatten() 

    return ee.FeatureCollection(cells).filterBounds(geom)#.map(lambda c: ee.Feature(c).intersection(geom) )

#### Ejemplo

In [259]:
#----------------------------------------------
# Seleccion del parque
#----------------------------------------------

# region
polygonCollection = 'WCMC/WDPA/current/polygons'
polygonName = 'Cerro Saroche'#'Canaima National Park'#'Guatopo'

# poligono
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))


# geometria
state = polygon.geometry()
dx = 0.0288
dy = 0.0288

# creando grilla
greed = generateGreedForGeometry(state,dx,dy)
centroide = state.centroid().coordinates().getInfo()[::-1]

def getCentroid(feature):
    keepProperties = ['name', 'huc6', 'tnmid', 'areasqkm']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)

#centroides
centroids = greed.map(getCentroid)

print('área del parque Km2: ', 0.1*polygon.first().geometry().area().divide(100*100).getInfo() ) 
print('área de cuadricula Km2: ', 0.1*greed.first().geometry().area().divide(100*100).getInfo() )
print('número de centroides: ', centroids.size().getInfo())

área del parque Km2:  3629.434345090513
área de cuadricula Km2:  100.9365181750428
número de centroides:  74


#### data ECMWF/ERA5/MONTHLY

In [260]:
#https://confluence.ecmwf.int/display/CKB/ERA5%3A+What+is+the+spatial+reference#ERA5:Whatisthespatialreference-5Gridgeometry
import numpy as np
from shapely.geometry import MultiLineString, MultiPoint
from shapely.ops import polygonize

dx = 0.25
dy = 0.25

# generando grilla
x = np.arange(-180.0 + dx, 180.0 + dx, dx)
y = np.arange(-90.0 + dy, 90.0 + dy, dy)

hlines = [((x1, yi), (x2, yi)) for x1, x2 in zip(x[:-1], x[1:]) for yi in y]
vlines = [((xi, y1), (xi, y2)) for y1, y2 in zip(y[:-1], y[1:]) for xi in x]

# grillas
grids = list(polygonize(MultiLineString(hlines + vlines)))

# creando poligonops
google_poligon = ee.Geometry(polygon.geometry()).bounds().coordinates().get(0).getInfo()
poligono = Polygon(list(map(lambda x: tuple(x), google_poligon)))

# interseccion
intersection = list(map(lambda y: y if poligono.intersection(y) else None,grids))

# poligonos en area
poligono_ECMWF = [x for x in list(filter(None,intersection))]

# creando lista de centroides
lista_ = ee.List([np.array(x.centroid.coords).tolist()[0] for x in poligono_ECMWF])
centroides_ECMWF = ee.FeatureCollection(lista_.map( lambda x: ee.Feature(ee.Geometry.Point(x,proj='EPSG:4326'),{'id':lista_.indexOf(x)}) ))

In [261]:
# Elevacion
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation).clip(polygon)

# Precipitacion total
ECMWF = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

# calculando el punto con la elevacion
reducer = ee.Reducer.median()
centroids_elevation = elevation.reduceRegions( ee.FeatureCollection(centroides_ECMWF), reducer.setOutputs(['elevation']), 90)

In [262]:
def ECMWF_data(imagen_collection = ECMWF,
                date = '2015-12-01',
                imagen_elevation = centroids_elevation):

    reducer = ee.Reducer.median()
    centroids_ECMWF= imagen_collection\
                        .filterDate(date)\
                        .map(lambda x: x.reduceRegions(imagen_elevation,reducer.setOutputs(['total_precipitation']),2))

    lista = centroids_ECMWF.first().getInfo()

    if lista is None:
        return None

    return list(map(lambda x,y:
                    {'date': y,
                    'id':int(x.get('id')),
                    'longitude':x.get('geometry').get('coordinates')[0],
                    'latitude':x.get('geometry').get('coordinates')[1],
                    'dewpoint_2m_temperature': x.get('properties').get('dewpoint_2m_temperature'),
                    'elevation': x.get('properties').get('elevation'),
                    'maximum_2m_air_temperature': x.get('properties').get('maximum_2m_air_temperature'),
                    'mean_2m_air_temperature': x.get('properties').get('mean_2m_air_temperature'),
                    'mean_sea_level_pressure': x.get('properties').get('mean_sea_level_pressure'),
                    'minimum_2m_air_temperature': x.get('properties').get('minimum_2m_air_temperature'),
                    'surface_pressure': x.get('properties').get('surface_pressure'),
                    'total_precipitation': x.get('properties').get('total_precipitation'),
                    'u_component_of_wind_10m': x.get('properties').get('u_component_of_wind_10m'),
                    'v_component_of_wind_10m': x.get('properties').get('v_component_of_wind_10m')
                    },
                    lista.get('features'),
                    [lista.get('properties').get('system:index') for x in range(len(lista.get('features'))) ]
                    ))

#ECMWF_data()

In [263]:
import pandas as pd
import numpy as np

# rango de meses
date_start="1900-01-01"
date_end="2021-06-01"

# consultando la data
data = np.array( list(filter(None,
                    list(map(lambda x: ECMWF_data(date=x)
                                    ,pd.date_range(start=date_start,end=date_end, closed='right',freq='M').strftime('%Y-%m').to_list()
                                    ))
                )))

data = data.reshape((
                    1,
                    data.shape[0]*data.shape[1]
                    )).tolist()[0]

data = pd.DataFrame(data)
data['year'] = pd.to_datetime(data['date'],format='%Y%m').dt.year
data['month'] = pd.to_datetime(data['date'],format='%Y%m').dt.month
data['date'] = data['date'].apply(int)
data['polygonName'] = polygonName

data.sort_values(['date','id'],ascending=True).head(30)

,date,id,longitude,latitude,dewpoint_2m_temperature,elevation,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m,year,month,polygonName
0,197901,0,-69.875,10.125,288.950317,447,309.641174,298.714264,101234.710938,291.081451,95410.210938,0.002376,-2.376098,-0.956133,1979,1,Cerro Saroche
1,197901,1,-69.875,10.375,288.983124,372,307.772980,296.900574,101297.007812,288.397919,93878.710938,0.004026,-2.214913,-0.627389,1979,1,Cerro Saroche
2,197901,2,-69.625,10.125,289.047058,768,308.392487,297.397430,101286.492188,290.337738,94179.265625,0.003298,-2.631367,-1.042131,1979,1,Cerro Saroche
3,197901,3,-69.625,10.375,289.597443,674,308.374725,296.777893,101312.156250,289.688995,94380.812500,0.006980,-2.438816,-0.692560,1979,1,Cerro Saroche
4,197901,4,-69.375,10.125,289.260071,676,306.667877,295.543976,101357.289062,288.829346,92788.359375,0.009322,-2.570262,-1.487913,1979,1,Cerro Saroche
5,197901,5,-69.375,10.375,290.359100,675,307.829437,296.038666,101336.640625,289.640686,94878.414062,0.010899,-2.394880,-0.900221,1979,1,Cerro Saroche
6,197902,0,-69.875,10.125,288.752197,447,309.487701,299.586731,101221.453125,290.833069,95404.312500,0.001073,-3.550768,-1.069947,1979,2,Cerro Saroche
7,197902,1,-69.875,10.375,288.739624,372,307.622223,297.829926,101293.164062,289.211334,93883.343750,0.001962,-3.230129,-0.453546,1979,2,Cerro Saroche
8,197902,2,-69.625,10.125,288.867157,768,307.891846,298.240692,101284.320312,289.791412,94185.765625,0.002507,-3.778696,-0.965066,1979,2,Cerro Saroche
9,197902,3,-69.625,10.375,289.397308,674,307.873322,297.694489,101315.250000,289.514435,94392.234375,0.003597,-3.334412,-0.597369,1979,2,Cerro Saroche


In [264]:
# save data pkl
data.to_pickle('./data/{polygonName}_data.pkl'.format(polygonName=polygonName))

In [265]:
# x and y given as array_like objects
import plotly.express as px

fig = px.scatter(data[data.date==201001],
y='latitude',
x='longitude',
color='total_precipitation',
title='Precipitación')
fig.show()

In [267]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')

Map.addLayer(slope, {min: 0, max: 60}, 'Slope')
Map.addLayer(greed,{'color':'#FFFFFF80'}, 'Grids')
Map.addLayer(centroids,{'color':'#000000'}, 'Centroids')
Map.addLayer(centroides_ECMWF,{'color':'#FF0000'}, 'centroides_ECMWF')

Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map